In [4]:
using Graphs
using GraphPlot
using AbstractTrees
using GraphRecipes
using Plots
default(size=(1000, 1000))

In [5]:
mutable struct Node1
    itemset # vd: ['a', 'b']
    # parent # node cha (itemset cha) của itemset hiện tại 
    children
    sup 

    # constructor
    Node1(its, chi = [], su = 0) = new(its, chi, su)
end

In [6]:
two_itemset = [Node1([1,2]), Node1([1,3])]
one_itemset = [Node1([1]), Node1([2]), Node1([3])]
root = Node1([nothing], one_itemset)

itr = root
itr.children[1].children = two_itemset

tree = root 


Node1([nothing], Node1[Node1([1], Node1[Node1([1, 2], Any[], 0), Node1([1, 3], Any[], 0)], 0), Node1([2], Any[], 0), Node1([3], Any[], 0)], 0)

In [7]:
AbstractTrees.children(node::Node1) = node.children
function AbstractTrees.printnode(io::IO, node::Node1)
    str = node.itemset
    print(io, str)
end
# AbstractTrees.printnode(io::IO, node::Node1) = print(io, "$(data)")
print_tree(tree)


[nothing]
├

─ 

[1]
│  ├─ [1, 2]
│  └─ [1, 3]
├─ [2]
└─ [3]


In [8]:
a = [1,2,3]
b = [1,2]

a[1:end-1] == b

true

# BACKUP !

In [ ]:
function apriori_tree(database, min_sup)
    one_itemset = countmap(collect(Iterators.flatten(database)))
    one_itemset = [Node([item.first], [], item.second) for item in one_itemset if item.second >= min_sup]
    root_node = Node([nothing], one_itemset)
    itr = root_node.children # root_node iterator

    while !isempty(itr)
        println("generating candidates...")
        candidate_nodes = candidate_nodes_gen(itr) # queue dc pass-by-ref -> update queue's children
        
        try
            println(length(candidate_nodes[1].itemset))
        catch
            # pass
        end
        
        for transaction in database
            for candidate in candidate_nodes 
                if candidate.itemset ⊆ transaction # if transaction contains candidate 
                    candidate.sup += 1
                end
            end
        end
        
        # lọc ra tập phổ biến, với các tập ko phổ biến thì xóa nó khỏi cây
        large_itemset = []
        for node in candidate_nodes
            if node.sup >= min_sup
                push!(large_itemset, node)
            else
                par = node.parent 
                deleteat!(par.children, findall(x->x==node, par.children))
                node.parent = []
            end
        end
        
        itr = large_itemset
    end

    println("Done")
    return root_node 
end

In [ ]:
function candidate_gen(large_itemset)
    candidate_list = Set()
    k = length(large_itemset[1]) + 1 # SL ptu của mỗi candidate trong candidate_list (hay Ck)

    # for (transaction i) and (transaction j) in large_itemset
    for i=1:length(large_itemset)
        for j=i+1:length(large_itemset)
            # 2 transactions phải giống nhau n-1 ptu, chỉ khác nhau bởi đúng 1 ptu
            u = union(Set(large_itemset[i]), Set(large_itemset[j]))
            # nếu thỏa mãn DK trên thì tập intersection chỉ dài hơn large_itemset[i] đúng 1 ptu
            if length(u) == k
                # tỉa nhánh: 
                flag = 0 
                u_converted = sort!((x->(x)).(u)) # convert u (set) to u (Array) 
                for subset in collect(combinations(u_converted,k-1))
                    sort!(subset) # VD: ['C','T'] ∈ [['A','B'], ['C', 'T']] trong khi  ['T','C'] ∉ [['A','B'], ['C', 'T']]
                    # nếu tồn tại tập con k-1 ptu không phổ biến (tức ko nằm trong large_itemset) thì tập cha chứa nó ko phổ biến -> ko thêm tập cha k ptu vào candidate_list
                    if subset ∉ large_itemset 
                        flag = 1
                        break
                    end
                end
                # ngược lại, nếu tất cả tập con phổ biến thì tập cha dc thêm vào candidate_list
                if flag == 0
                    push!(candidate_list, u_converted)
                end
            end
        end
    end
    return candidate_list
end

In [ ]:
function candidate_nodes_gen(large_itemset_nodes)
    large_itemset = [node.itemset for node in large_itemset_nodes]
    candidate_nodes = []
    candidate_list = Set()
    k = length(large_itemset[1]) + 1 # SL ptu của mỗi candidate trong candidate_list (hay Ck)
    loopi = 1

    # for (transaction i) and (transaction j) in large_itemset
    for i=1:length(large_itemset)
        for j=i+1:length(large_itemset)
            # 2 transactions phải giống nhau n-1 ptu, chỉ khác nhau bởi đúng 1 ptu
            u = union(Set(large_itemset[i]), Set(large_itemset[j]))
            # nếu thỏa mãn DK trên thì tập union chỉ dài hơn large_itemset[i] đúng 1 ptu
            if length(u) == k   
                # tỉa nhánh: 
                flag = 0 
                # sort u to guarantee a lexico order, then convert u (Set) to u (Array) 
                u_converted = sort!((x->(x)).(u)) 

                for subset in collect(combinations(u_converted,k-1))
                    sort!(subset) # VD: ['C','T'] ∈ [['A','B'], ['C','T']] trong khi  ['T','C'] ∉ [['A','B'], ['C','T']]
                    # nếu tồn tại tập con k-1 ptu không phổ biến (tức ko nằm trong large_itemset) thì tập cha chứa nó ko phổ biến -> ko thêm tập cha k ptu vào candidate_list
                    if subset ∉ large_itemset 
                        flag = 1
                        break
                    end
                end
                # ngược lại, nếu tất cả tập con phổ biến thì tập cha dc thêm vào candidate_list
                if flag == 0 && u_converted ∉ candidate_list
                    push!(candidate_list, u_converted)

                    node = Node(u_converted)
                    node.parent = large_itemset_nodes[i]
                    push!(large_itemset_nodes[i].children, node) # append children to parent node
                    push!(candidate_nodes, node)
                end
            end
        end  
    end
    return candidate_nodes
end